<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/stats_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime

In [3]:

drive.mount('/content/drive')
     

Mounted at /content/drive


In [5]:
Data_basic = pd.read_csv("/content/drive/MyDrive/play_ID_data.csv")

In [30]:
Data_basic

,GameID,season,types,week,HomeTeam_ID,AwayTeam_ID,Winner
0,401326322,2021,2,1,27,6,Home
1,401326315,2021,2,1,1,21,Away
2,401326308,2021,2,1,2,23,Away
3,401326316,2021,2,1,4,16,Home
4,401326317,2021,2,1,8,25,Away
...,...,...,...,...,...,...,...
539,401437957,2022,2,18,21,19,Home
540,401437962,2022,2,18,25,22,Home
541,401437963,2022,2,18,26,14,Home
542,401437959,2022,2,18,28,6,Home


In [28]:
play_time=list(range(0,901,60))

In [69]:
stats_df = pd.DataFrame()
stats_df = pd.DataFrame(columns=['GameID','Pass_Incompletion_Home',
  'Punt_Home',
  'Timeout_Home',
  'Punt_Away',
  'Pass_Incompletion_Away',
  'Timeout_Away',
  'homeScore',
  'awayScore',
  'PassReception_Home_yards',
  'PassReception_Away_yards',
  'Penalty_Home',
  'Penalty_Away',
  'Penalty_Home_yards',
  'Penalty_Away_yards',
  'rushing_yards_home',
  'rushing_yards_away',
  'field_goals_good_home',
  'field_goals_good_away',
  'field_goals_missed_home',
  'field_goals_missed_away',
  'fumble_recovery_oponnent_home',
  'fumble_recovery_oponnent_away',
  'fumble_recovery_own_home',
  'fumble_recovery_own_away',
  'sack_yards_home',
  'sack_yards_away',
  'passing_TD_home',
  'passing_TD_away',
  'rushing_TD_home',
  'rushing_TD_away',
  'interception_home',
  'interception_away',
  'Interception_Return_TD_home',
  'Interception_Return_TD_away',
  'kick_off_return_TD_home',
  'kick_off_return_TD_away',
  'safety_home',
  'safety_away'])

In [59]:
def stats(GameID,Quarter,time):
  url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
  response_drive = r.get(url)
  drive_data = response_drive.json()
  drive_df = json_normalize(drive_data['items'])
  data=[]
  play_clock = datetime.timedelta(seconds=time)
  if Quarter  == 1:
    time_left = datetime.timedelta(seconds=3600) - play_clock
  elif Quarter == 2:
    time_left = datetime.timedelta(seconds=2700) - play_clock
  elif Quarter == 3:
    time_left = datetime.timedelta(seconds=1800) - play_clock
  else:
    time_left = datetime.timedelta(seconds=900) - play_clock
  Pass_Incompletion_Home=0
  Punt_Home=0
  Timeout_Home=0
  Punt_Away=0
  Pass_Incompletion_Away=0
  Timeout_Away=0
  homeScore=0
  awayScore=0
  PassReception_Home_yards=0
  PassReception_Away_yards=0
  Penalty_Home=0
  Penalty_Away=0
  Penalty_Home_yards=0
  Penalty_Away_yards=0
  rushing_yards_home=0
  rushing_yards_away=0
  field_goals_good_home=0
  field_goals_good_away=0
  field_goals_missed_home=0
  field_goals_missed_away=0
  fumble_recovery_oponnent_home=0
  fumble_recovery_oponnent_away=0
  fumble_recovery_own_home=0
  fumble_recovery_own_away=0
  sack_yards_home=0
  sack_yards_away=0
  passing_TD_home=0
  passing_TD_away=0
  rushing_TD_home=0
  rushing_TD_away=0
  interception_home=0
  interception_away=0
  Interception_Return_TD_home=0
  Interception_Return_TD_away=0
  kick_off_return_TD_home=0
  kick_off_return_TD_away=0
  safety_home=0
  safety_away=0
  for index, row in drive_df.iterrows():
    try:
      if (re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]) &(row['start.period.number']==Quarter):
        for detail in range(len(row['plays.items'])):
          #duration_str = row['plays.items'][detail]['clock']['displayValue']
          #print(duration_str)
          #duration_arr = duration_str.split(":")
          #duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))
          #duration=datetime.timedelta(seconds=900)-duration
          #if duration <= datetime.timedelta(seconds=time):
            if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
              Pass_Incompletion_Home=Pass_Incompletion_Home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
              Punt_Home=Punt_Home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
              Timeout_Home=Timeout_Home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
              PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
              Penalty_Home_yards=Penalty_Home_yards+row['plays.items'][detail]['statYardage']
              Penalty_Home=Penalty_Home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
              rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
              sack_yards_home=row['plays.items'][detail]['statYardage']+sack_yards_home
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
              passing_TD_home=passing_TD_home+1
              PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
              rushing_TD_home=rushing_TD_home+1
              rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
              interception_home=interception_home+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
              field_goals_good_home=field_goals_good_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
              field_goals_missed_home=field_goals_missed_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
              Interception_Return_TD_home=Interception_Return_TD_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
              kick_off_return_TD_home=kick_off_return_TD_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
              fumble_recovery_own_home=fumble_recovery_own_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
              fumble_recovery_oponnent_home=fumble_recovery_oponnent_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
              safety_home=safety_home+1
            homeScore=row['plays.items'][detail]['homeScore']
      elif (re.search(r"teams/(\d+)",row['team.$ref']).group(1) != Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]) &(row['start.period.number']==Quarter):
        for detail in range(len(row['plays.items'])):
          #duration_str2 = row['plays.items'][detail]['clock']['displayValue']
          #duration_arr2 = duration_str2.split(":")
          #duration2 = datetime.timedelta(minutes=int(duration_arr2[0]), seconds=int(duration_arr2[1]))
          #duration2=datetime.timedelta(seconds=900)-duration2
          #if duration2 <= datetime.timedelta(seconds=time):
            if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
              Pass_Incompletion_Away=Pass_Incompletion_Away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
              Punt_Away=Punt_Away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
              Timeout_Away=Timeout_Away+1  
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
              PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
              Penalty_Away_yards=Penalty_Away_yards+row['plays.items'][detail]['statYardage']
              Penalty_Away=Penalty_Away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
              rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
              #print(row['plays.items'][detail]['id'])
              #print(row['plays.items'][detail]['statYardage'])
              sack_yards_away=row['plays.items'][detail]['statYardage']+sack_yards_away
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
              passing_TD_away=passing_TD_away+1
              PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
              rushing_TD_away=rushing_TD_away+1
              rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
              interception_away=interception_away+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
              field_goals_good_away=field_goals_good_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
              field_goals_missed_away=field_goals_missed_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
              Interception_Return_TD_away=Interception_Return_TD_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
              kick_off_return_TD_away=kick_off_return_TD_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
              fumble_recovery_own_away=fumble_recovery_own_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
              fumble_recovery_oponnent_away=fumble_recovery_oponnent_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
              safety_away=safety_away+1
            awayScore=row['plays.items'][detail]['awayScore']
    except TypeError:
      print('it doenst worked')
  data = [    GameID,    play_clock, time_left,Pass_Incompletion_Home,
          Punt_Home,    Timeout_Home,
          Punt_Away,    Pass_Incompletion_Away,
          Timeout_Away,    homeScore,
          awayScore,    PassReception_Home_yards,
          PassReception_Away_yards,    Penalty_Home,
          Penalty_Away,    Penalty_Home_yards,
          Penalty_Away_yards,    rushing_yards_home,
          rushing_yards_away,    field_goals_good_home,
          field_goals_good_away,    field_goals_missed_home,
          field_goals_missed_away,    fumble_recovery_oponnent_home,
          fumble_recovery_oponnent_away,    fumble_recovery_own_home,
          fumble_recovery_own_away,    sack_yards_home,
          sack_yards_away,    passing_TD_home,
          passing_TD_away,    rushing_TD_home,
          rushing_TD_away,    interception_home,
          interception_away,    Interception_Return_TD_home,
          Interception_Return_TD_away,    kick_off_return_TD_home,
          kick_off_return_TD_away,
          safety_home,    safety_away]
  stats_df.loc[len(stats_df)] = data


In [67]:
def stats(GameID,Quarter):
  url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
  response_drive = r.get(url)
  drive_data = response_drive.json()
  drive_df = json_normalize(drive_data['items'])
  data=[]
  Pass_Incompletion_Home=0
  Punt_Home=0
  Timeout_Home=0
  Punt_Away=0
  Pass_Incompletion_Away=0
  Timeout_Away=0
  homeScore=0
  awayScore=0
  PassReception_Home_yards=0
  PassReception_Away_yards=0
  Penalty_Home=0
  Penalty_Away=0
  Penalty_Home_yards=0
  Penalty_Away_yards=0
  rushing_yards_home=0
  rushing_yards_away=0
  field_goals_good_home=0
  field_goals_good_away=0
  field_goals_missed_home=0
  field_goals_missed_away=0
  fumble_recovery_oponnent_home=0
  fumble_recovery_oponnent_away=0
  fumble_recovery_own_home=0
  fumble_recovery_own_away=0
  sack_yards_home=0
  sack_yards_away=0
  passing_TD_home=0
  passing_TD_away=0
  rushing_TD_home=0
  rushing_TD_away=0
  interception_home=0
  interception_away=0
  Interception_Return_TD_home=0
  Interception_Return_TD_away=0
  kick_off_return_TD_home=0
  kick_off_return_TD_away=0
  safety_home=0
  safety_away=0
  for index, row in drive_df.iterrows():
    try:
      if (re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]) &(row['start.period.number']==Quarter):
        for detail in range(len(row['plays.items'])):
          if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
            Pass_Incompletion_Home=Pass_Incompletion_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
            Punt_Home=Punt_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
            Timeout_Home=Timeout_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
            PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
            Penalty_Home_yards=Penalty_Home_yards+row['plays.items'][detail]['statYardage']
            Penalty_Home=Penalty_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
            rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
            sack_yards_home=row['plays.items'][detail]['statYardage']+sack_yards_home
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
            passing_TD_home=passing_TD_home+1
            PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
            rushing_TD_home=rushing_TD_home+1
            rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
            interception_home=interception_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
            field_goals_good_home=field_goals_good_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
            field_goals_missed_home=field_goals_missed_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
            Interception_Return_TD_home=Interception_Return_TD_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
            kick_off_return_TD_home=kick_off_return_TD_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
            fumble_recovery_own_home=fumble_recovery_own_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
            fumble_recovery_oponnent_home=fumble_recovery_oponnent_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
            safety_home=safety_home+1
          homeScore=row['plays.items'][detail]['homeScore']
      elif (re.search(r"teams/(\d+)",row['team.$ref']).group(1) != Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]) &(row['start.period.number']==Quarter):
        for detail in range(len(row['plays.items'])):
          if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
            Pass_Incompletion_Away=Pass_Incompletion_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
            Punt_Away=Punt_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
            Timeout_Away=Timeout_Away+1  
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
            PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
            Penalty_Away_yards=Penalty_Away_yards+row['plays.items'][detail]['statYardage']
            Penalty_Away=Penalty_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
            rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
            #print(row['plays.items'][detail]['id'])
            #print(row['plays.items'][detail]['statYardage'])
            sack_yards_away=row['plays.items'][detail]['statYardage']+sack_yards_away
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
            passing_TD_away=passing_TD_away+1
            PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
            rushing_TD_away=rushing_TD_away+1
            rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
            interception_away=interception_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
            field_goals_good_away=field_goals_good_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
            field_goals_missed_away=field_goals_missed_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
            Interception_Return_TD_away=Interception_Return_TD_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
            kick_off_return_TD_away=kick_off_return_TD_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
            fumble_recovery_own_away=fumble_recovery_own_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
            fumble_recovery_oponnent_away=fumble_recovery_oponnent_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
            safety_away=safety_away+1
          awayScore=row['plays.items'][detail]['awayScore']
    except TypeError:
      print('it doenst worked')
  data = [    GameID,    Pass_Incompletion_Home,
          Punt_Home,    Timeout_Home,
          Punt_Away,    Pass_Incompletion_Away,
          Timeout_Away,    homeScore,
          awayScore,    PassReception_Home_yards,
          PassReception_Away_yards,    Penalty_Home,
          Penalty_Away,    Penalty_Home_yards,
          Penalty_Away_yards,    rushing_yards_home,
          rushing_yards_away,    field_goals_good_home,
          field_goals_good_away,    field_goals_missed_home,
          field_goals_missed_away,    fumble_recovery_oponnent_home,
          fumble_recovery_oponnent_away,    fumble_recovery_own_home,
          fumble_recovery_own_away,    sack_yards_home,
          sack_yards_away,    passing_TD_home,
          passing_TD_away,    rushing_TD_home,
          rushing_TD_away,    interception_home,
          interception_away,    Interception_Return_TD_home,
          Interception_Return_TD_away,    kick_off_return_TD_home,
          kick_off_return_TD_away,
          safety_home,    safety_away]
  stats_df.loc[len(stats_df)] = data







In [70]:
stats(401326322,1)

In [71]:
stats_df

,GameID,Pass_Incompletion_Home,Punt_Home,Timeout_Home,Punt_Away,Pass_Incompletion_Away,Timeout_Away,homeScore,awayScore,PassReception_Home_yards,...,rushing_TD_home,rushing_TD_away,interception_home,interception_away,Interception_Return_TD_home,Interception_Return_TD_away,kick_off_return_TD_home,kick_off_return_TD_away,safety_home,safety_away
0,401326322,0,0,0,4,7,0,0,7,0,...,0,0,0,0,0,0,0,0,0,0


<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class